In [1]:
# === CÀI ĐẶT THƯ VIỆN ===
!pip install easyocr paddleocr paddlepaddle face_recognition opencv-python-headless matplotlib
!apt install -y libgl1-mesa-glx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 10.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.8/192.8 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 128.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 69.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [7]:
# === IMPORT ===
import cv2
import numpy as np
import easyocr
import face_recognition
import matplotlib.pyplot as plt
from google.colab import files
from PIL import Image
from paddleocr import PaddleOCR
import os

# === UPLOAD ẢNH THẺ SV ===
print(" Vui lòng upload ảnh thẻ sinh viên (jpg/png)")
uploaded = files.upload()
image_path = next(iter(uploaded))  # tên file ảnh vừa upload

# === ĐỌC ẢNH ===
def read_image(path):
    img = cv2.imread(path)
    if img is None:
        raise ValueError("Không thể đọc ảnh.")
    return img

image = read_image(image_path)

# === OCR: EasyOCR + PaddleOCR ===
print(" Đang kiểm tra ảnh có phải thẻ sinh viên đúng mẫu không...")

reader = easyocr.Reader(['vi', 'en'])
result_easy = reader.readtext(image_path)

ocr = PaddleOCR(use_angle_cls=True, lang='vi')
result_paddle = ocr.ocr(image_path, cls=True)[0]

# Gộp kết quả từ 2 OCR
texts = [r[1] for r in result_easy]
texts += [line[1][0] for line in result_paddle]

text_all = ' '.join(texts).upper()
print("📑 Văn bản trích xuất từ ảnh:\n", text_all)

# === KIỂM TRA TỪ KHÓA ===
if "THẺ SINH VIÊN" in text_all or "SINH VIÊN" in text_all or "THANG LONG UNIVERSITY" in text_all:
    print(" Đây có vẻ là thẻ sinh viên hợp lệ.")
    valid_card = True
else:
    print(" Không phát hiện dấu hiệu là thẻ sinh viên.")
    valid_card = False

# === TÁCH KHUÔN MẶT TỪ ẢNH THẺ ===
if valid_card:
    face_locations = face_recognition.face_locations(image)
    if len(face_locations) == 0:
        print(" Không tìm thấy khuôn mặt trong ảnh thẻ.")
        valid_card = False
    else:
        face_enc_card = face_recognition.face_encodings(image)[0]
        print(" Đã trích xuất khuôn mặt từ ảnh thẻ.")

# === CHỤP HOẶC UPLOAD ẢNH THẬT ===
if valid_card:
    print(" Upload ảnh khuôn mặt thực tế để so sánh (jpg/png)")
    uploaded_face = files.upload()
    real_face_path = next(iter(uploaded_face))
    image_real = read_image(real_face_path)

    face_locations_real = face_recognition.face_locations(image_real)
    if len(face_locations_real) == 0:
        print("Không phát hiện khuôn mặt trong ảnh thực tế.")
    else:
        face_enc_real = face_recognition.face_encodings(image_real)[0]

        # So sánh
        result = face_recognition.compare_faces([face_enc_card], face_enc_real)
        distance = face_recognition.face_distance([face_enc_card], face_enc_real)[0]

        print(f" Khoảng cách khuôn mặt: {distance:.2f}")
        if result[0]:
            print(" Khuôn mặt khớp. Đây là chủ nhân hợp lệ của thẻ sinh viên.")
        else:
            print(" Khuôn mặt không khớp với ảnh trên thẻ.")

        # Hiển thị 2 ảnh
        fig, axs = plt.subplots(1, 2, figsize=(10, 4))
        axs[0].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        axs[0].set_title("Ảnh thẻ sinh viên")
        axs[1].imshow(cv2.cvtColor(image_real, cv2.COLOR_BGR2RGB))
        axs[1].set_title("Ảnh khuôn mặt thực tế")
        for ax in axs: ax.axis('off')
        plt.show()

 Vui lòng upload ảnh thẻ sinh viên (jpg/png)


Saving IMG_2052.jpg to IMG_2052 (1).jpg
 Đang kiểm tra ảnh có phải thẻ sinh viên đúng mẫu không...
[2025/05/12 12:18:56] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, use_gcu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/root/.paddleocr/whl/rec/latin/lat